Based on:
https://github.com/ml4a/ml4a-guides/blob/master/notebooks/recurrent_neural_networks.ipynb

## Recurrent Neural Networks: Character RNNs with Keras

Often we are not interested in isolated datapoints, but rather datapoints within a context of others. A datapoint may mean something different depending on what's come before it. This can typically be represented as some kind of _sequence_ of datapoints, perhaps the most common of which is a time series.

One of the most ubiquitous sequences of data where context is especially important is natural language. We have quite a few words in English where the meaning of a word may be totally different depending on it's context. An innocuous example of this is "bank": "I went fishing down by the river bank" vs "I deposited some money into the bank".

If we consider that each word is a datapoint, most non-recurrent methods will treat "bank" in the first sentence exactly the same as "bank" in the second sentence - they are indistinguishable. If you think about it, in isolation they are indistinguishable to us as well - it's the same word!

We can only start to discern them when we consider the previous word (or words). So we might want our neural network to consider that "bank" in the first sentence is preceded by "river" and that in the second sentence "money" comes a few words before it. That's basically what RNNs do - they "remember" some of the previous context and that influences the output it produces. This "memory" (called the network's "_hidden state_") works by retaining some of the previous outputs and combining it with the current input; this recursing (feedback) of the network's output back into itself is where its name comes from.

This recursing makes RNNs quite deep, and thus they can be difficult to train. The gradient gets smaller and smaller the deeper it is pushed backwards through the network until it "vanishes" (effectively becomes zero), so long-term dependencies are hard to learn. The typical practice is to only extend the RNN back a certain number of time steps so the network is still trainable.

Certain units, such as the LSTM (long short-term memory) and GRU (gated recurrent unit), have been developed to mitigate some of this vanishing gradient effect.

Let's walkthrough an example of a character RNN, which is a great approach for learning a character-level language model. A language model is essentially some function which returns a probability over possible words (or in this case, characters), based on what has been seen so far. This function can vary from region to region (e.g. if terms like "pop" are used more commonly than "soda") or from person to person. You could say that a (good) language model captures the style in which someone writes.

Language models often must make the simplifying assumption that only what came immediately (one time step) before matters (this is called the "Markov assumption"), but with RNNs we do not need to make such an assumption.

We'll use Keras which makes building neural networks extremely easy (this example is an annotated version of Keras's [LSTM text generation example](https://github.com/fchollet/keras/blob/master/examples/lstm_text_generation.py)).

First we'll do some simple preparation - import the classes we need and load up the text we want to learn from.

In [1]:
import os

#if using Theano with GPU
#os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu,floatX=float32"

import random
import numpy as np
from glob import glob
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Activation, Dropout
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm

import pandas as pd

Using TensorFlow backend.


In [2]:
%matplotlib inline
from matplotlib import pyplot as plt

In [3]:
# load up our text
all_jokes = pd.read_csv('shortjokes.csv')
all_jokes.head()

,ID,Joke
0,1,"[me narrating a documentary about narrators] ""..."
1,2,Telling my daughter garlic is good for you. Go...
2,3,I've been going through a really rough period ...
3,4,"If I could have dinner with anyone, dead or al..."
4,5,Two guys walk into a bar. The third guy ducks.


In [4]:
# let us create a long string variable text
text = '\n'.join(all_jokes.Joke)
text[0:500]

'[me narrating a documentary about narrators] "I can\'t hear what they\'re saying cuz I\'m talking"\nTelling my daughter garlic is good for you. Good immune system and keeps pests away.Ticks, mosquitos, vampires... men.\nI\'ve been going through a really rough period at work this week It\'s my own fault for swapping my tampax for sand paper.\nIf I could have dinner with anyone, dead or alive... ...I would choose alive. -B.J. Novak-\nTwo guys walk into a bar. The third guy ducks.\nWhy can\'t Barbie get pregn'

In [5]:
# extract all (unique) characters
# these are our "categories" or "labels". We want to predict the next character from the past few (e.g 20) characters

def removeChars(text):
    for char in ['\x08', '\x10', '~', '^']:
        text = text.replace(char, '')
    return text
  
all_jokes.Joke = all_jokes.Joke.apply(lambda x: removeChars(x))

text = '\n'.join(all_jokes.Joke)
text[0:500]

chars = list(set(text))

all_jokes.Joke = all_jokes.Joke.apply(lambda x: '~' + x + '^')
print(sorted(chars))

['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}']


Now we'll define our RNN. Keras makes this trivial:

In [6]:
# model = Sequential()
# model.add(LSTM(512, return_sequences=True, input_shape=(max_len, len(chars))))
# model.add(Dropout(0.2))
# model.add(LSTM(512, return_sequences=False))
# model.add(Dropout(0.2))
# model.add(Dense(len(chars)))
# model.add(Activation('softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [7]:
# model.summary()

We're framing our task as a classification task. Given a sequence of characters, we want to predict the next character. We equate each character with some label or category (e.g. "a" is 0, "b" is 1, etc).

We use the _softmax_ activation function on our output layer - this function is used for categorical output. It turns the output into a probability distribution over the categories (i.e. it makes the values the network outputs sum to 1). So the network will essentially tell us how strongly it feels about each character being the next one.

The categorical cross-entropy loss the standard loss function for multilabel classification, which basically penalizes the network more the further off it is from the correct label.

We use dropout here to prevent overfitting - we don't want the network to just return things already in the text, we want it to have some wiggle room and create novelty! Dropout is a technique where, in training, some percent (here, 20%) of random neurons of the associated layer are "turned off" for that epoch. This prevents overfitting by preventing the network from relying on particular neurons.

That's it for the network architecture!

To train, we have to do some additional preparation. We need to chop up the text into character sequences of the length we specified (`max_len`) - these are our training inputs. We match them with the character that immediately follows each sequence. These are our expected training outputs.

For example, say we have the following text (this quote is from Zhuang Zi). With `max_len=20`, we could manually create the first couple training examples like so:

We also need to map each character to a label and create a reverse mapping to use later:

In [8]:
char_labels = {ch:i+3 for i, ch in enumerate(chars)}
labels_char = {i+3:ch for i, ch in enumerate(chars)}
# Padding Char:
char_labels['PAD'] = 0
labels_char[0] = 'PAD'
# Start Char
char_labels['~'] = 1
labels_char[1] = '~'
# End Char
char_labels['^'] = 2
labels_char[2] = '^'

In [9]:
z = sorted(list(char_labels.values()))
for i, k in enumerate(z):
    if i != k:
        print(i)

In [10]:
print(char_labels)

{'3': 3, 'O': 4, 'Y': 5, 'F': 6, '}': 7, 'N': 8, 'V': 9, '-': 10, 'd': 11, 'X': 12, 'r': 13, 'w': 14, 's': 15, '4': 16, '0': 17, "'": 18, '(': 19, '_': 20, ' ': 21, 'E': 22, 'M': 23, 'L': 24, 'k': 25, 'D': 26, '>': 27, 'i': 28, 'x': 29, '{': 30, 'I': 31, '"': 32, 'h': 33, 'j': 34, 'W': 35, 'T': 36, '$': 37, '`': 38, 'a': 39, '|': 40, 'R': 41, ';': 42, '<': 43, 'o': 44, 'Q': 45, ']': 46, 'K': 47, ':': 48, '=': 49, 't': 50, 'c': 51, '2': 52, 'g': 53, '.': 54, 'A': 55, 'q': 56, '\\': 57, '5': 58, '9': 59, 'U': 60, '[': 61, 'y': 62, '?': 63, 'f': 64, 'b': 65, '#': 66, 'u': 67, '*': 68, '+': 69, '8': 70, '/': 71, 'C': 72, 'z': 73, ')': 74, 'e': 75, '6': 76, 'G': 77, 'H': 78, 'p': 79, 'P': 80, 'B': 81, 'S': 82, '&': 83, '1': 84, '%': 85, 'Z': 86, ',': 87, 'm': 88, '\n': 89, 'J': 90, 'n': 91, 'v': 92, '@': 93, 'l': 94, '7': 95, '!': 96, 'PAD': 0, '~': 1, '^': 2}


Now we can start constructing our numerical input 3-tensor and output matrix. Each input example (i.e. a sequence of characters) is turned into a matrix of one-hot vectors; that is, a bunch of vectors where the index corresponding to the character is set to 1 and all the rest are set to zero.

For example, if we have the following:

In [11]:
print(len(char_labels))

97


In [12]:
### Creating X Data
max_len = 200
x_data = np.array([np.array([char_labels[char] for char in z[:-1]]) for z in all_jokes.Joke])
# input_sequences = np.array(pad_sequences(x_data,   
#                             maxlen=max_len, padding='post'))


y_data = np.array([np.array([char_labels[char] for char in z[1:]]) for z in all_jokes.Joke])
# true_output = np.array(pad_sequences(y_data,   
#                             maxlen=max_len, padding='post'))


# seq_lengths = np.array([len(z) for z in all_jokes.Joke])
# max_len = 20
# x_data = [[char_labels[char] for char in z] for z in all_jokes.Joke]
# input_sequences = []
# labels = []
# for seq in tqdm(x_data):
#   for i in range(len(seq)):
#     if i+max_len < len(seq):
#       input_sequences.append(seq[i:i+max_len])
#       labels.append(seq[i+max_len])

In [13]:
x_data.shape, y_data.shape

((231657,), (231657,))

In [14]:
import tensorflow as tf
from keras.utils import to_categorical

class CharRNN:
    def __init__(self):
        self.global_step = tf.train.get_or_create_global_step()
        self.build_model()


    def build_model(self):
        with tf.variable_scope('Initialization'):
            self.x = tf.placeholder(tf.int32, shape=[None, None], name='Features')
            self.y = tf.placeholder(tf.int32, shape=[None, None], name='Labels')
            self.seq_lengths = tf.placeholder(tf.int32, shape=[None,], name='Sequence_Lengths')
            
            char_embeddings = tf.get_variable("char_embeddings", [98, 36])
            embedded_char_ids = tf.nn.embedding_lookup(char_embeddings, self.x)

        with tf.variable_scope('LSTM'):
            lstm1 = tf.contrib.cudnn_rnn.CudnnCompatibleLSTMCell(num_units=256)
#             lstm2 = tf.contrib.cudnn_rnn.CudnnCompatibleLSTMCell(num_units=256)
            
            lstm1_output, _ = lstm1(tf.transpose(embedded_char_ids, [1,0,2]))
#             lstm2_output, _ = lstm2(lstm1_output)

            logits = tf.layers.dense(lstm1_output, units=97+1)
            logits = tf.transpose(logits, [1,0,2])

        with tf.variable_scope('Optimization'):
            optimizer = tf.train.AdamOptimizer(0.001)
            loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=self.y)

            self.loss = tf.reduce_mean(loss)
            self.train_step = optimizer.minimize(self.loss, global_step=self.global_step)
            
        with tf.variable_scope('Decoding'):
            start_toks = tf.fill([1], 1)
            initial_state = tf.zeros(shape=[36,256])
            decoder = tf.contrib.seq2seq.BeamSearchDecoder(cell=lstm1, 
                                              embedding=char_embeddings,
                                              start_tokens=start_toks,
                                              end_token=2,
                                              initial_state=tf.zeros([128, 256], dtype=tf.float32),
                                              beam_width=10)
            
            decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
                                                          decoder = decoder,
                                                          impute_finished = False,
                                                          maximum_iterations = 200)
            
  
def padDataBatch(data, labels):
    seq_lengths = np.array([utter.shape[0] for utter in data])
    maxlen = max(seq_lengths)
    x_data = np.array([np.pad(utter, ((0, maxlen-utter.shape[0])), mode='constant') for utter in data], dtype="int32")
    y_data = np.array([np.pad(utter, ((0, maxlen-utter.shape[0])), mode='constant') for utter in labels], dtype="int32")
    to_categorical(x_data, num_classes=97)
    return x_data, seq_lengths, y_data
    
    
    
def train(sess, model, num_epochs, input_seq, output_seq):
    batch_size = 128
    data_size = input_seq.shape[0]
    all_idxs = np.arange(data_size)

    for k in range(num_epochs):
        np.random.shuffle(all_idxs)
        i = 0
        losses = []
        for i in tqdm(range(0, len(all_idxs), batch_size)):
            idxs = all_idxs[i:i+batch_size]

            batch_data, batch_lens, batch_y = padDataBatch(input_seq[idxs], output_seq[idxs])
            loss = sess.run([model.loss], feed_dict={model.x:batch_data, model.y:batch_y, 
                                                   model.seq_lengths:batch_lens})
            losses.append(loss)
            
            
        print(np.mean(losses))
        
  

In [15]:
model = CharRNN()

saver = tf.train.Saver(max_to_keep=10)
chkpt = tf.train.get_checkpoint_state('logs/')
saver_hook = tf.train.CheckpointSaverHook(checkpoint_dir='logs/', save_secs=100, saver=saver)
with tf.train.SingularMonitoredSession(hooks=[saver_hook]) as sess:
    loss = train(sess, model, 10, x_data, y_data)

Instructions for updating:
Colocations handled automatically by placer.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



TypeError: __call__() missing 1 required positional argument: 'state'